<center><h1>QUOTEMINER!! concept</h1></center>


Concept and testing notebook file for QuoteMiner. Cos jupyter variables really do come in handy cuh

In [1]:
import os
import whisperx
import subprocess
import pathlib
import torch
import json
from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from typing import List
import dotenv
from concurrent.futures import ProcessPoolExecutor

import time
from typing import Optional

/Users/itzmaniss/coding/QuoteMiner/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/itzmaniss/coding/QuoteMiner/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


In [1]:
from faster_whisper import WhisperModel

/Users/itzmaniss/coding/QuoteMiner/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def transcribeAudio(audio_path):
    try:
        print("Transcribing audio...")
        Device = "cuda" if torch.cuda.is_available() else "cpu"
        print(Device)
        model = WhisperModel(
            "base.en", device="cuda" if torch.cuda.is_available() else "cpu"
        )
        print("Model loaded")
        segments, info = model.transcribe(
            audio=audio_path,
            beam_size=5,
            language="en",
            max_new_tokens=128,
            condition_on_previous_text=False,
        )
        segments = list(segments)
        # print(segments)
        extracted_texts = [
            [segment.text, segment.start, segment.end] for segment in segments
        ]
        return extracted_texts
    except Exception as e:
        print("Transcription Error:", e)
        return []

In [4]:
x = transcribeAudio("../videos/short.mp4")

Transcribing audio...
cpu


[2025-07-22 01:38:18.797] [ctranslate2] [thread 4320935] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.


Model loaded


In [7]:
with open("sample_transcription.json", "w") as f:
    json.dump({"transcription": x}, f, indent=4)

In [6]:
import json

In [2]:
from pathlib import Path

output_folder = Path("./videos/")


f"{output_folder}/%(title)s.%(ext)s"

'videos/%(title)s.%(ext)s'

note: Should thread the youtube download script to not waste time. only network heavy


In [ ]:
# link = "https://www.youtube.com/watch?v=hBMoPUAeLnY"
link = "https://www.youtube.com/watch?v=-CJI-xeVp9c"
subprocess.run(
    [
        "yt-dlp",
        "-f",
        "bv*[ext=wav]+ba",
        "--merge-output-format",
        "wav",
        "-o",
        "./videos/%(title)s.%(ext)s",
        link,
        "--progress",
    ]
)  # -"a", "your_links.txt"

[youtube] Extracting URL: https://www.youtube.com/watch?v=-CJI-xeVp9c
[youtube] -CJI-xeVp9c: Downloading webpage
[youtube] -CJI-xeVp9c: Downloading tv client config
[youtube] -CJI-xeVp9c: Downloading player 14cfd4c0-main
[youtube] -CJI-xeVp9c: Downloading tv player API JSON
[youtube] -CJI-xeVp9c: Downloading ios player API JSON
[youtube] -CJI-xeVp9c: Downloading m3u8 information
[info] -CJI-xeVp9c: Downloading 1 format(s): 401+251
[download] Destination: ./videos/My Father Demanded I Attend My Golden Brother's Wedding, Threatening To Cut....- Reddit Stories.f401.mp4
[download] 100% of    1.89GiB in 00:02:51 at 11.26MiB/s     
[download] Destination: ./videos/My Father Demanded I Attend My Golden Brother's Wedding, Threatening To Cut....- Reddit Stories.f251.webm
[download] 100% of   17.01MiB in 00:00:01 at 10.85MiB/s    
[Merger] Merging formats into "./videos/My Father Demanded I Attend My Golden Brother's Wedding, Threatening To Cut....- Reddit Stories.mp4"
Deleting original file ./v

CompletedProcess(args=['yt-dlp', '-f', 'bv*[ext=mp4]+ba', '--merge-output-format', 'mp4', '-o', './videos/%(title)s.%(ext)s', 'https://www.youtube.com/watch?v=-CJI-xeVp9c', '--progress'], returncode=0)

In [5]:
dotenv.load_dotenv()
# Defining constants

input_dir = pathlib.Path("./videos")
output_dir = pathlib.Path("./clips")
transcriptions_dir = pathlib.Path("./transcriptions")
hf_token = os.getenv("hugging_face_token")
# set if you have local whisperx model
model_path = None


# default pytorch variables
device = "cpu"
# batch_size =
compute_type = "int8"
video_path = pathlib.Path("./videos/short.mp4")

In [3]:
# Check for existing directories and make if needed

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(transcriptions_dir, exist_ok=True)

In [9]:
# video_path = pathlib.Path("/Users/itzmaniss/coding/QuoteMiner/videos/Andrew Tate vs Therapist, Round #1. [2ELmm02-jCQ].mp4")
video_path = pathlib.Path(
    "/Users/itzmaniss/coding/QuoteMiner/videos/My Father Demanded I Attend My Golden Brother's Wedding, Threatening To Cut....- Reddit Stories.mp4"
)

In [10]:
# Step 1: Transcription
# def transcribe(video_path: pathlib.Path):
print("Loading MOdels\n\n")
model = whisperx.load_model("large-v2", device=device, compute_type="int8")
audio = whisperx.load_audio(str(video_path.absolute()))
print("Transcribing\n\n")
result = model.transcribe(audio, language="en", batch_size=8)
print("Segmenting\n\n")
complete_trancsript = ""
for segment in result["segments"]:
    complete_trancsript += segment["text"]

print("Aligning\n\n")
model_a, metadata = whisperx.load_align_model(
    language_code=result["language"], device=device
)
refined_result = whisperx.align(
    result["segments"], model_a, metadata, audio, "cpu", return_char_alignments=False
)
word_timestamps = []
# Iterate through each segment to extract word timestamps and confidence scores
print("Word Scoring\n\n")
for segment in refined_result["segments"]:
    for word_info in segment.get("words", []):
        if "start" in word_info and "end" in word_info:
            word_timestamps.append(
                {
                    "word": word_info["word"],
                    "start": word_info["start"],  # Assuming start and end are present
                    "end": word_info["end"],  # Assuming start and end are present
                    "score": word_info.get("score", 0),  # Default score if not provided
                }
            )

print("Storing Transcription Data\n\n")
transript_path = transcriptions_dir / (video_path.stem + ".json")
with open(transript_path, "w") as file:
    json.dump({"text": complete_trancsript, "word_timestampls": word_timestamps}, file)

Loading MOdels




Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint .venv/lib/python3.9/site-packages/whisperx/assets/pytorch_model.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1. Bad things might happen unless you revert torch to 1.x.
Transcribing


Segmenting


Aligning


Word Scoring


Storing Transcription Data




In [4]:
from llama_index.core import Document

with open("transcriptions/Joe Rogan Experience #2223 - Elon Musk.json", "r") as file:
    data = json.load(file)

x = Document(text=data["text"])

In [ ]:
# def extract_quotes(transcriptions_dir: pathlib.Path) -> list:
Settings.llm = Ollama(model="deepseek-r1:14b")
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# documents = SimpleDirectoryReader(input_dir=transcriptions_dir).load_data()
index = VectorStoreIndex.from_documents(
    [
        x,
    ]
)

with open("prompt.txt", "r") as file:
    detailed_prompt = file.read()


query_engine = index.as_query_engine()
response = query_engine.query(detailed_prompt)

y = response.response

quotes = json.loads(
    remove_trailing_commas(y.strip().replace("```", "").strip().replace("json", ""))
)

with open("motivational_lines.json", "w") as output_file:
    output_file.write(response.response)

    # return quotes

In [ ]:
y = response.response
n = remove_trailing_commas(y.strip().replace("```", "").strip().replace("json", ""))
json.loads(n)

In [30]:
y = response.response

In [ ]:
y

In [ ]:
remove_trailing_commas(y.strip().replace("```", "").strip().replace("json", ""))

In [ ]:
quotes = json.loads(
    remove_trailing_commas(y.strip().replace("```", "").strip().replace("json", ""))
)

with open("motivational_lines.txt", "w") as output_file:
    output_file.write(response.response)

In [ ]:
with open("prompt.txt", "r") as file:
    detailed_prompt = file.read()


query_engine = index.as_query_engine()
response = query_engine.query(detailed_prompt)

In [13]:
import re


def remove_trailing_commas(json_string):
    """
    Remove trailing commas from a JSON string.

    Args:
        json_string (str): The JSON string to clean.

    Returns:
        str: The JSON string without trailing commas.
    """
    # Regex to detect trailing commas before a closing brace or bracket
    trailing_comma_pattern = re.compile(r",\s*([}\]])")

    # Replace all trailing commas with the closing brace/bracket
    cleaned_json = trailing_comma_pattern.sub(r"\1", json_string)

    return cleaned_json

In [5]:
def generate_clips(video_path: pathlib.Path, quotes: List[dict]) -> None:
    with ProcessPoolExecutor() as exexutor:
        for idx, quote in enumerate(quotes):
            exexutor.submit(generate_clip, quote, idx, video_path)

In [4]:
def generate_clip(quote: dict, idx: int, video_path: pathlib.Path) -> None:
    start = quote["start"]
    end = quote["end"]
    if start is None or end is None:
        print(f"Skipping quote {idx + 1} due to missing timestamps.")
        return

    output_file = output_dir / f"{video_path.stem}_{idx + 1}.mp4"
    cmd = [
        "ffmpeg",
        "-i",
        video_path,
        "-ss",
        str(start),
        "-to",
        str(end),
        "-c",
        "copy",
        output_file,
    ]
    subprocess.run(cmd)

In [ ]:
generate_clips()

In [19]:
def setup_llm_settings(retries: int = 3, timeout: int = 300) -> None:
    """
    Configure the LLM settings with retry logic and extended timeout.

    Args:
        retries: Number of times to retry on failure
        timeout: Timeout in seconds for the Ollama request
    """
    Settings.llm = Ollama(
        model="deepseek-r1:14b",
        timeout=600,
        request_timeout=600,  # Adding explicit request timeout
        temperature=0.7,  # Adding temperature to help with response generation
        context_window=4096,  # Explicitly setting context window
    )

    Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


def process_llm_response(response_text: str) -> dict:
    """
    Clean and parse the LLM response into JSON.

    Args:
        response_text: Raw response from the LLM

    Returns:
        Parsed JSON dictionary
    """
    cleaned_text = response_text.strip()
    cleaned_text = cleaned_text.replace("```json", "").replace("```", "")
    cleaned_text = cleaned_text.replace("json", "").strip()
    return json.loads(remove_trailing_commas(cleaned_text))


def extract_quotes(
    document, prompt_path: pathlib.Path, max_retries: int = 3
) -> Optional[dict]:
    """
    Extract quotes from the document with retry logic.

    Args:
        document: Document to analyze
        prompt_path: Path to the prompt file
        max_retries: Maximum number of retry attempts

    Returns:
        Dictionary of extracted quotes or None if all retries fail
    """
    # Setup LLM with increased timeout
    setup_llm_settings()

    # Create index from document
    index = VectorStoreIndex.from_documents([document])

    # Read prompt
    with open(prompt_path, "r") as file:
        detailed_prompt = file.read()

    # Initialize query engine
    query_engine = index.as_query_engine()

    # Attempt query with retries
    for attempt in range(max_retries):
        try:
            response = query_engine.query(detailed_prompt)
            print(response)
            quotes = process_llm_response(response.response)

            # Save response to file
            with open("motivational_lines.json", "w") as output_file:
                json.dump(quotes, output_file, indent=2)

            return quotes

        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Failed after {max_retries} attempts. Final error: {str(e)}")
                return None

            print(f"Attempt {attempt + 1} failed. Retrying after delay...")
            time.sleep(2**attempt)  # Exponential backoff

    return None

In [ ]:
quotes = extract_quotes(x, pathlib.Path("prompt.txt"))
if quotes:
    print("Successfully extracted quotes")
else:
    print("Failed to extract quotes")

In [19]:
from src.timestamp_utils import precompute_word_indices, find_best_fit_optimized


def find_timestamps(quotes, transcript_path):
    with open(transcript_path, "r") as file:
        transcript_data = json.load(file)
    word_to_indices, transcript_words = precompute_word_indices(transcript_data["text"])
    timestamped_quotes = []
    for quote in quotes:
        try:
            start, end = find_best_fit_optimized(
                quotes[quote],
                transcript_data["word_timestamps"],
                transcript_words,
                word_to_indices,
            )
            timestamped_quotes.append(
                {"text": quotes[quote], "start": start, "end": end}
            )
        except ValueError as e:
            print(f"Could not find timestamps for quote: {quote}. Error: {e}")
    return timestamped_quotes

In [20]:
import json
import pathlib

with open("motivational_lines.json", "r") as file:
    quotes = json.load(file)
file = pathlib.Path(
    "/Users/itzmaniss/coding/QuoteMiner/transcriptions/Joe Rogan Experience #2219 - Donald Trump.json"
)
timestamped_quotes = find_timestamps(quotes, file)

In [ ]:
from utils.video_processing import generate_clips

video_file = pathlib.Path(
    "/Users/itzmaniss/coding/QuoteMiner/videos/Joe Rogan Experience #2219 - Donald Trump.mp4"
)
OUTPUT_DIR = pathlib.Path("./clips")
generate_clips(video_file, timestamped_quotes, OUTPUT_DIR)

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

In [21]:
timestamped_quotes

[{'text': 'Nobody gives up prime time for being president. They told me I was crazy. The Apprentice was still a hit, NBC wanted me to stay, but I walked away because I knew I had to do this.',
  'start': 135.379,
  'end': 147.889},
 {'text': 'The moment I started winning, everything changed. The same people who once praised me, hugged me, and called me a friend turned against me overnight. It wasn’t just politics—it was the machine at work.',
  'start': 248.801,
  'end': 259.863},
 {'text': 'When I got shot, it wasn’t surreal. When I became president, that was surreal. Lying on the ground, I knew exactly what was happening. But standing in the White House for the first time, looking around, realizing I was president—that was something else entirely.',
  'start': 306.329,
  'end': 320.22},
 {'text': 'You don’t think about the weight of the office until you’re standing in the Lincoln Bedroom, looking at the Gettysburg Address, knowing you now hold the responsibility of history in your ha

In [ ]:
with open("debug_response.txt", "r") as f:
    message = f.read()

In [9]:
import anthropic
import dotenv
import os
import json
from typing import List, Optional
from models import Quote

dotenv.load_dotenv()


class QuoteExtractorConfig:
    def __init__(
        self,
        model: str = "claude-3-haiku-20240307",
        max_tokens: int = 4096,
        temperature: float = 0.7,
        max_clips: int = 3,
        max_duration: int = 60,
    ):
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.max_clips = max_clips
        self.max_duration = max_duration


class QuoteExtractor:
    def __init__(self, config: Optional[QuoteExtractorConfig] = None):
        self.config = config or QuoteExtractorConfig()
        self.client = self._initialize_client()
        self.system_prompt = self._build_system_prompt()

    def _initialize_client(self) -> anthropic.Anthropic:
        api_key = os.getenv("ANTHROPIC_API_KEY")
        if not api_key:
            raise ValueError("ANTHROPIC_API_KEY environment variable not set")
        return anthropic.Anthropic(api_key=api_key)

    def _build_system_prompt(self) -> str:
        return f"""You are a motivational clip extractor.
The user provides a list of transcribed text segments, each with a start and end timestamp.
Your task is to select one or more non-overlapping, high-impact, motivational moments that could be used as TikTok-style short clips.
✦ Each clip must:
Be less than {self.config.max_duration} seconds in duration.
Be contiguous — use consecutive lines from the input with no skips.
Be non-overlapping with other clips (start/end times must not intersect).
Be motivational or emotionally engaging (thought-provoking, passionate, punchy).

✦ Output Format: Only return valid JSON in this structure:
[
  {{
    "start": "Start time of the clip (in seconds)",
    "content": "Motivational quote from the transcript",
    "end": "End time of the clip (in seconds)"
  }},
  ...
]

Clip count: 1 to {self.config.max_clips} clips max.
Do not include any explanation, introduction, or text before/after the JSON.
If no suitable clips are found, return an empty array: [].

✦ Reminder (Grave Consequence Clause):
If any output overlaps, exceeds {self.config.max_duration} seconds, is not contiguous, or breaks JSON syntax, 10 kittens will die and the JSON parser will explode.
So format perfectly and choose wisely."""

    def extract_quotes(self, transcription: list) -> List[Quote]:
        if not transcription:
            raise ValueError("Transcription cannot be empty")

        try:
            message = self.client.messages.create(
                model=self.config.model,
                max_tokens=self.config.max_tokens,
                temperature=self.config.temperature,
                system=self.system_prompt,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": str(transcription),
                            }
                        ],
                    }
                ],
            )

            # return self._parse_response(message.content)
            return message

        except Exception as e:
            raise RuntimeError(f"Failed to extract quotes: {str(e)}")

    def _parse_response(self, response) -> List[Quote]:
        try:
            response_text = (
                response[0].text
                if hasattr(response, "__iter__") and response
                else str(response)
            )
            quotes_data = json.loads(response_text)

            if not isinstance(quotes_data, list):
                raise ValueError("Response must be a list")

            quotes = []
            for quote_data in quotes_data:
                quote = Quote(
                    start=float(quote_data["start"]),
                    content=quote_data["content"],
                    end=float(quote_data["end"]),
                )
                self._validate_quote(quote)
                quotes.append(quote)

            self._validate_no_overlaps(quotes)
            return quotes

        except json.JSONDecodeError:
            raise ValueError("Invalid JSON response from API")
        except (KeyError, ValueError, TypeError) as e:
            raise ValueError(f"Invalid quote format: {str(e)}")

    def _validate_quote(self, quote: Quote) -> None:
        if quote.start >= quote.end:
            raise ValueError(
                f"Invalid time range: start {quote.start} >= end {quote.end}"
            )

        if quote.duration > self.config.max_duration:
            raise ValueError(
                f"Quote duration {quote.duration}s exceeds max {self.config.max_duration}s"
            )

        if not quote.content.strip():
            raise ValueError("Quote content cannot be empty")

    def _validate_no_overlaps(self, quotes: List[Quote]) -> None:
        for i, quote1 in enumerate(quotes):
            for quote2 in quotes[i + 1 :]:
                if quote1.start < quote2.end and quote2.start < quote1.end:
                    raise ValueError(
                        f"Overlapping quotes detected: {quote1.start}-{quote1.end} and {quote2.start}-{quote2.end}"
                    )

In [10]:
extractor = QuoteExtractor()
try:
    with open("sample_transcription.json", "r") as f:
        transcription = json.load(f)["transcription"]
    message = extractor.extract_quotes(transcription)
#     for quote in quotes:
#         print(json.dumps(quote.to_dict(), indent=2))
except Exception as e:
    print(f"Error: {e}")

In [11]:
type(message)

anthropic.types.message.Message

In [13]:
y = extractor._parse_response(message.content)

In [14]:
y

[Quote(start=49.44, content="there's gonna be the great honey conspiracy of like 2022.", end=58.239999999999995),
 Quote(start=69.68, content="but there's some shady shit going on.", end=72.04),
 Quote(start=209.96, content="Cuz there's some shady shit happening. I don't trust it. I don't trust it", end=215.6)]

In [15]:
for quote in y:
    print(json.dumps(quote.to_dict(), indent=2))

{
  "start": "49.44",
  "content": "there's gonna be the great honey conspiracy of like 2022.",
  "end": "58.239999999999995"
}
{
  "start": "69.68",
  "content": "but there's some shady shit going on.",
  "end": "72.04"
}
{
  "start": "209.96",
  "content": "Cuz there's some shady shit happening. I don't trust it. I don't trust it",
  "end": "215.6"
}


ModuleNotFoundError: No module named 'moviepy.editor'